<a href="https://colab.research.google.com/github/ko-i/study/blob/main/tensorflow2/code48.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
raw_inputs = [
    [711, 632, 71],
    [73, 8, 3215, 55, 927],
    [83, 91, 1, 645, 1253, 927],
]

padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(raw_inputs, padding='post')
print(padded_inputs)

In [ ]:
embedding = layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True)
masked_output = embedding(padded_inputs)
print(masked_output._keras_mask)

In [ ]:
masking_layer = layers.Masking()
unmasked_embedding = tf.cast(tf.tile(tf.expand_dims(padded_inputs, axis=-1), [1, 1, 10]), tf.float32)

masked_embedding = masking_layer(unmasked_embedding)
print(masked_embedding._keras_mask)

In [ ]:
model = keras.Sequential([
    layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True),
    layers.LSTM(32)
])

In [ ]:
inputs = keras.Input(shape=(None, ), dtype='int32')
x = layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True)(inputs)
outputs = layers.LSTM(32)(x)

model = keras.Model(inputs, outputs)

In [ ]:
class MyLayer(layers.Layer):
    def __init__(self, **kwargs):
        super(MyLayer, self).__init__(**kwargs)
        self.embedding = layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True)
        self.lstm = layers.LSTM(32)
    
    def call(self, inputs):
        x = self.embedding(inputs)

        mask = self.embedding.compute_mask(inputs)
        output = self.lstm(x, mask=mask)

        return output

layer = MyLayer()
x = np.random.random((32, 10)) * 100
x = x.astype('int32')
layer(x)

In [ ]:
class TemporalSplit(keras.layers.Layer):
    def call(self, inputs):
        return tf.split(inputs, 2, axis=1)
    
    def compute_mask(self, inputs, mask):
        if mask is None:
            return None
        return tf.split(mask, 2, axis=1)

first_half, second_half = TemporalSplit()(masked_embedding)
print(first_half._keras_mask)
print(second_half._keras_mask)

In [ ]:
class CustomEmbedding(layers.Layer):
    def __init__(self, input_dim, output_dim, mask_zero=False, **kwargs):
        super(CustomEmbedding, self).__init__(**kwargs)
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.mask_zero = mask_zero
    
    def build(self, input_shape):
        self.embeddings = self.add_weight(shape=(self.input_dim, self.output_dim), initializer='random_normal', dtype='float32')

    def call(self, inputs):
        return tf.nn.embedding_lookup(self.embeddings, inputs)
    
    def compute_mask(self, inputs, mask=None):
        if not self.mask_zero:
            return None
        
        return tf.not_equal(inputs, 0)

layer = CustomEmbedding(10, 32, mask_zero=True)
x = np.random.random((3, 10)) * 9
x = x.astype('int32')

y = layer(x)
mask = layer.compute_mask(x)
print(mask)

In [ ]:
class MyActivation(keras.layers.Layer):
    def __init__(self, **kwargs):
        super(MyActivation, self).__init__(**kwargs)
        self.supports_masking = True
    
    def call(self, inputs):
        return tf.nn.relu(inputs)

In [ ]:
inputs = keras.Input(shape=(None, ), dtype='int32')
x = layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True)(inputs)
x = MyActivation()(x)
print('mask found: ', x._keras_mask)
outputs = layers.LSTM(32)(x)

model = keras.Model(inputs, outputs)

In [ ]:
class TemporalSoftmax(keras.layers.Layer):
    def call(self, inputs, mask=None):
        broadcast_float_mask = tf.expand_dims(tf.cast(mask, 'float32'), -1)
        inputs_exp = tf.exp(inputs) * broadcast_float_mask
        inputs_sum = tf.reduce_sum(inputs_exp * broadcast_float_mask, axis=-1, keepdims=True)
        return inputs_exp / inputs_sum

inputs = keras.Input(shape=(None,), dtype='int32')
x = layers.Embedding(input_dim=10, output_dim=32, mask_zero=True)(inputs)
x = layers.Dense(1)(x)
outputs = TemporalSoftmax()(x)
model = keras.Model(inputs, outputs)

y = model(np.random.randint(0, 10, size=(32, 100)), np.random.random((32, 100, 1)))